# EIE 1st Year Project - Reference Design 2019

This notebook demonstrates the interface between PYNQ's Processing Systems (PS) and a custom HLS hardware block integrated with PYNQ's base overlay for real-time Video Processing. It implements a Vertical-Edge detector capable of running in >30fps. 

This material can be used for academic purposes only. Any commerical use is prohibited. 

Contact: Alexandros Kouris (a.kouris16@imperial.ac.uk), Ph.D. Candidate, Imperial College London

In [0]:
from pynq import Overlay, Xlnk
from pynq.overlays.base import BaseOverlay

allocator = Xlnk()

#Load customised version of Base-Overlay,that included the custom hardware block, instead of using BaseOverlay("base.bit")
#------------------------------------------------------------------------------------------------------------------------
#ol = Overlay("eie_v25.bit") #Grayscale, full-frame (33fps)
#ol = Overlay("eie_v25_2.bit") #Grayscale, half-frame (33fps)
ol = Overlay("e3.bit") #Vertical Edge-Detector, half-frame (33fps)  - EIE2019

In [0]:
from pynq.lib.video import *



hdmi_in = ol.video.hdmi_in
hdmi_out = ol.video.hdmi_out

hdmi_in.configure(PIXEL_RGBA)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGBA)

In [0]:
hdmi_in.start()
hdmi_out.start()



In [0]:

from PIL import Image
sourceFileName = "test.png"
avatar = Image.open(sourceFileName)
avatar.show()

In [0]:
from pynq import MMIO
#filter_reference = MMIO(0x00000000,0x10000)
filter_reference = MMIO(0x83C20000,0x10000)  #MMIO Addresses should always be double-checked when exporting a Vivado Design
add_io = MMIO(0x83C30000,0x10000)

In [0]:
import time

start = time.time()

#assign value

x_width=60;
#package x_pillar SEED = 0x2504705906AA8CA9CF8B9BFA;
x_pillar_i_0=0x2C0C8000
x_pillar_i_1=0x02581901
x_pillar_i_2=0x4B03E832
interval = 200;
v_horizontal = 50;

s_bird = 100;
y_bird =210;

#package y_depth SEED = 0x0AA0AA0AA0AA0AA0AA0AA0AA;
y_depth_i_0 =0xC8064570
y_depth_i_1 =0x419012C0
y_depth_i_2 =0x2BC2581F
new_depth = 100;
gap = 300;

r_avg_i =162;
g_avg_i =111;
b_avg_i =97;
error =50;
case_t = 50;
max_h=1
leftbound = 500
rightbound = 680

game_over = 0


for _ in range(1200):
    game_over = 0
    
    if _ < 250:
        case_t=1 #pink show
        y_bird = 350
        
    elif _ < 300:
        case_t=2 #green pass
        y_bird = 350
    elif _ < 500:
        y_bird = 350
        case_t=15 #transparent no collision
    else:
        case_t=30
      
   
    in_frame = hdmi_in.readframe()
    out_frame = hdmi_out.newframe()
    filter_reference.write(0x10, in_frame.physical_address)   # Get Pointers to memory
    filter_reference.write(0x18, out_frame.physical_address)
    filter_reference.write(0x20, 1280)   # Make sure that the input HDMI signal is set to 1280x720
    filter_reference.write(0x28, 720)
    
   
    #filter_reference.write(0x20, in_frame.shape[1]) 
    #filter_reference.write(0x28, in_frame.shape[0])
    add_io.write(0x10, x_width)
    add_io.write(0x18, x_pillar_i_0)
    add_io.write(0x1c, x_pillar_i_1)
    add_io.write(0x20, x_pillar_i_2)

    add_io.write(0x38, interval)
    add_io.write(0x40, v_horizontal)
    add_io.write(0x48, s_bird)
    add_io.write(0x50, y_bird)
  
    add_io.write(0x60, y_depth_i_0)
    add_io.write(0x64, y_depth_i_1)
    add_io.write(0x68, y_depth_i_2)

    add_io.write(0x80, new_depth)
    add_io.write(0x88, gap)
    add_io.write(0x90, game_over)
    
    add_io.write(0xa0, r_avg_i)
    add_io.write(0xb0, g_avg_i)
    add_io.write(0xc0, b_avg_i)

    add_io.write(0xd0, case_t)
    add_io.write(0xd8, error)
    add_io.write(0xe0, max_h)
    add_io.write(0xf0, leftbound)
    add_io.write(0xf8, rightbound)
    
    
    filter_reference.write(0x00, 0x01)             # ap_start triggering
    
    while (filter_reference.read(0) & 0x4) == 0:   # ap_done checking
        pass
      
    #Read and Store
    if add_io.read(0x34) == 1:
        x_pillar_i_0=add_io.read(0x28)
        x_pillar_i_1=add_io.read(0x2c)
        x_pillar_i_2=add_io.read(0x30)
    
    
    if add_io.read(0x7c) == 1:
        y_depth_i_0 =add_io.read(0x70)
        y_depth_i_1 =add_io.read(0x74)
        y_depth_i_2 =add_io.read(0x78)
  
    if add_io.read(0xac) == 1:
        r_avg_i =add_io.read(0xa8)
    if add_io.read(0xbc) == 1:
        g_avg_i =add_io.read(0xb8)
    if add_io.read(0xcc) == 1:
        b_avg_i =add_io.read(0xc8)
    
    if add_io.read(0xec) == 1:
        max_h = add_io.read(0xe8)
    
    if add_io.read(0x9c) == 1:
        game_over=add_io.read(0x98)
    
    hdmi_out.writeframe(out_frame)

end = time.time()

print(f"600 frames took {end-start} seconds at {1200/(end-start)} fps")

if add_io.read(0x5c) == 1:
        y_bird =add_io.read(0x58)-100
    if y_bird > 600:
        y_bird = y_bird-400
    elif y_bird > 500:
        y_bird = y_bird-350
    elif y_bird > 400:
        y_bird = y_bird-300
    elif y_bird > 300:
        y_bird = y_bird-250
    elif y_bird > 200:
        y_bird = y_bird-200

In [0]:
hdmi_in.close()    # Don't forget to run this to free memory
hdmi_out.close()   # NEVERFORGET NEVERFORGET NEVERFORGET :p 